<a href="https://colab.research.google.com/github/zizilnam/Mini_Project_Machine_Learning_Elo_Merchant_Kaggle/blob/main/Elo_project_Final_blending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Blending

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd/content/drive/MyDrive/KDT/모델링 프로젝트/data/data_1

/content/drive/MyDrive/KDT/모델링 프로젝트/data/data_1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold

from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
import pickle

from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score, log_loss

from sklearn.metrics import mean_squared_error
from math import sqrt

import time
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Data Load

In [ ]:
train = reduce_mem_usage(pd.read_csv('/content/drive/MyDrive/KDT/모델링 프로젝트/data/train_features_generated.csv', index_col=0))
test = reduce_mem_usage(pd.read_csv('/content/drive/MyDrive/KDT/모델링 프로젝트/data/test_features_generated.csv', index_col=0))

Mem. usage decreased to 100.71 Mb (74.7% reduction)
Mem. usage decreased to 61.19 Mb (74.7% reduction)


In [ ]:
with open('low_prob_test_predictions.pkl', 'rb') as f:
  test_low_pred = pickle.load(f)

with open('low_prob_train_predictions.pkl', 'rb') as f:
  train_low_pred = pickle.load(f)

with open('high_prob_train_predictions.pkl', 'rb') as f:
  train_high_pred = pickle.load(f)

with open('high_prob_test_predictions.pkl', 'rb') as f:
  test_high_pred = pickle.load(f)

#Load the full regression predictions

with open('train_predictions_full_regression.pkl', 'rb') as f:
  full_regression_train_preds = pickle.load(f)

with open('test_predictions_full_regression.pkl', 'rb') as f:
  full_regression_test_preds = pickle.load(f)

In [ ]:
train = train.merge(train_low_pred, on = 'card_id', how='left')
train = train.merge(train_high_pred, on = 'card_id', how='left')
train = train.merge(full_regression_train_preds, on = 'card_id', how='left')

test = test.merge(test_low_pred, on = 'card_id', how='left')
test = test.merge(test_high_pred, on = 'card_id', how='left')
test = test.merge(full_regression_test_preds, on = 'card_id', how='left')

In [ ]:
train_target = train['target']
test_card_id = test['card_id']

In [ ]:
columns = ['low_prob_score', 'high_prob_score', 'full_regression_prediction']
train = train[columns]
test = test[columns]

## Full Blend Model

BayesianRegression model 으로 모델링 시도합니다.

두번째는 RandomforestRegressor: 3.79391600433

세번째 Linear Regression: 3.68191541424833

네번째 sgdr : 3.96

Random Forest Regressor Parameter Tuning

In [ ]:
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
params = {'alpha': alpha}

model=SGDRegressor()
random_search = RandomizedSearchCV(model, params, scoring = 'neg_root_mean_squared_error', cv = 5)
random_search.fit(train, train_target)
random_search.best_params_

{'alpha': 0.0001}

In [ ]:
folds_stack = KFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(train.shape[0])
test_pred = np.zeros(test.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train,train_target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train.iloc[trn_idx], train_target.iloc[trn_idx]
    val_data, val_y = train.iloc[val_idx], train_target.iloc[val_idx]
    
    clf_3 = SGDRegressor(alpha=0.0001)
    clf_3.fit(trn_data, trn_y)
    
    oof[val_idx] = clf_3.predict(val_data)
    test_pred += clf_3.predict(test) / 5
    
print("CV RMSE:", np.sqrt(mean_squared_error(train_target.values, oof)))

fold 0
fold 1
fold 2
fold 3
fold 4
CV RMSE: 3.7196374501784955


## Kaggle Submission

In [ ]:
kaggle_submission = pd.DataFrame()
kaggle_submission['card_id'] = test_card_id
kaggle_submission['target'] = test_pred

In [ ]:
kaggle_submission

,card_id,target
0,C_ID_0ab67a22ab,-0.388070
1,C_ID_130fd0cbdd,1.353149
2,C_ID_b709037bc5,0.586925
3,C_ID_d27d835a9f,1.072608
4,C_ID_2b5e3df5c2,0.454365
...,...,...
123618,C_ID_7a239d2eda,2.506311
123619,C_ID_75ace375ae,0.989641
123620,C_ID_21d56d950c,2.174796
123621,C_ID_6c46fc5a9d,-1.316626


In [ ]:
kaggle_submission.to_csv('final_blended_model_sgdr2.csv', index=0)